# Сбор данных с сайта РИА Новости


Установим и импортируем необходимые библиотеки.

In [1]:
%%capture
!pip install selenium

In [2]:
import re
import time
import datetime
import pandas as pd
import warnings
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass

warnings.filterwarnings("ignore", category=FutureWarning)

Далее зададим необходимые константы.

`DEPTH` - число страниц для скрапинга

In [3]:
SLEEP = 2
DEPTH = 20
BASE_URL = "https://ria.ru/"
TOPICS = ["economy", "society", "incidents", "science", "culture"]

In [4]:
@dataclass
class Article:
    id: str = None
    url: str = None
    title: str = None
    subtitle: str = None
    content: str = None
    datetime: str = None

In [5]:
# set webdriver params
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("headless")
chrome_options.add_argument("no-sandbox")
chrome_options.add_argument("disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

Посмотрим на функцию для скачивания страниц.

In [6]:
def get_pages():

    """Load and scroll pages"""

    items, topics_order = [], []

    for topic in tqdm(TOPICS):
        try:
            old_size = len(items)
            URL = BASE_URL + topic
            driver.get(URL)
            time.sleep(SLEEP)

            # push to list 20 next articles
            driver.execute_script(
                "document.getElementsByClassName('list-more')[0].click()"
            )
            time.sleep(1)

            # scroll page to automatically load more articles
            for i in tqdm(range(DEPTH), leave=False):
                try:
                    driver.execute_script(
                        f"window.scrollTo(0, document.body.scrollHeight - 1200)"
                    )
                    time.sleep(1)
                except:
                    pass

            # find all pages
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            scope = soup.find(
                "div", {"class": "list", "itemtype": "http://schema.org/ItemList"}
            )
            items += scope.find_all("div", {"class": "list-item"})

            # number of pages can not be multiple of deepth*20
            # that's why we count topics_order dynamically
            new_size = len(items)
            if new_size > old_size:
                topics_order.extend([topic] * (new_size - old_size))
        except:
            pass

    return items, topics_order

Посмотрим на функцию для парсинга скачанных страниц.

In [7]:
def parse_page(page):
    """Extract from page desired fields"""

    # Create article data class object
    article = Article()

    # article url
    article.url = page.find("a", {"class": "list-item__image"})["href"]

    # article id
    s = re.findall(r"\d+.html", article.url)[0]
    article.id = s[: s.find(".")]

    # load page
    driver.get(article.url)
    time.sleep(SLEEP)
    html = driver.page_source

    # article source
    source = article.url[8 : article.url.find(".")]

    # article object
    soup = BeautifulSoup(html, "html.parser")
    obj = soup.find(
        "div",
        {
            "class": lambda x: x and (x.find(f"article m-article m-{source}") > -1),
            "data-article-id": article.id,
        },
    )

    if not obj:
        obj = soup.find(
            "div",
            {
                "class": lambda x: x and (x.find(f"article m-video m-{source}") > -1),
                "data-article-id": article.id,
            },
        )

    # process article title
    title = obj.find("div", {"class": "article__title"})
    title_2 = obj.find("h1", {"class": "article__title"})

    if title:
        article.title = title.text
    else:
        article.title = title_2.text if title_2 else ""

    # article subtitle
    subtitle = obj.find("h1", {"class": "article__second-title"})
    article.subtitle = subtitle.text if subtitle else ""

    # article content
    article.content = obj.find(
        "div", {"class": "article__body js-mediator-article mia-analytics"}
    ).text

    # article datetime
    article.datetime = obj.find("div", {"class": "article__info-date"}).find("a").text

    # article number of views
    # article.views = int(obj.find('span', {'class': 'statistic__item m-views'}).text)

    return article

Скачаем новости (количество регулируется константой DEPTH выше).

In [8]:
# get pages and topics
pages, topics_order = get_pages()

100%|██████████| 5/5 [02:06<00:00, 25.24s/it]


Сохраним результат в файл.

In [9]:
with open("ria_pages.txt", "w", encoding="utf-8") as f:
    for page in pages:
        f.write("%s\n" % page)

## Вопрос 1

Сколько страниц `pages` мы спарсили?

In [10]:
len(pages)

1848

Распарсим каждую страницу и достанем из нее необходимые атрибуты:

* в цикле примените функцию `parse_page` к каждой странице и сохраните результат в объект `res`

* затем добавьте `res` в список `data`


Для скорости распарсим только первые 100 страниц.

In [11]:
data, topics_order_fixed = [], []

for num, page in enumerate(tqdm(pages[:100])):
    try:
        res = parse_page(page)
        data.append(res)

        topics_order_fixed.append(topics_order[num])
    except:
        pass

driver.close()

100%|██████████| 100/100 [05:07<00:00,  3.07s/it]


Преобразуем в датафрейм полученные данные.

## Вопрос 2

Сколько столбцов в таблице `df`?

In [13]:
df = pd.DataFrame(data=data)
df

,id,url,title,subtitle,content,datetime
0,1921570252,https://ria.ru/20240116/pivo-1921570252.html,В ГД оценили идею запрета ввоза импортного пива,В ГД назвали идею запрета ввоза импортного пив...,"МОСКВА, 16 янв – РИА Новости. Депутат Госдумы ...",11:17 16.01.2024
1,1921562909,https://ria.ru/20240116/pivo-1921562909.html,В Госдуме предложили запретить ввоз пива из не...,"""Известия"": в РФ хотят запретить поставки пива...","МОСКВА, 16 янв – РИА Новости. Глава комитета Г...",10:40 16.01.2024
2,1921490766,https://ria.ru/20240116/neft-1921490766.html,Голод не тетка: CША накупили российской нефти ...,США возобновили импорт нефти из России после п...,"МОСКВА, 16 янв — РИА Новости, Наталья Дембинск...",08:00 16.01.2024
3,1921539497,https://ria.ru/20240116/bortprovodniki-1921539...,"""Аэрофлот"" и ""Победа"" готовы принять на работу...","""Аэрофлот"" и ""Победа"" готовы принять на работу...","МОСКВА, 16 янв - РИА Новости. Крупнейшая росси...",07:24 16.01.2024
4,1921529559,https://ria.ru/20240116/platina-1921529559.html,Германия сократила закупки платины у России до...,ФРГ в ноябре сократила закупки платины у Росси...,"МОСКВА, 16 янв - РИА Новости. Германия к концу...",03:05 16.01.2024
...,...,...,...,...,...,...
94,1920886637,https://ria.ru/20240112/minpromtorg-1920886637...,Минпромторг разработал проект эксперимента по ...,Минпромторг: в России предложили маркировать р...,"МОСКВА, 12 янв - РИА Новости. Минпромторг Росс...",11:23 12.01.2024
95,1920878136,https://ria.ru/20240112/moldaviya-1920878136.html,"Додон заявил, что власти Молдавии все равно по...",Додон: власти Молдавии покупают тот же российс...,"КИШИНЕВ, 12 янв – РИА Новости. Экс-президент М...",10:56 12.01.2024
96,1920866381,https://ria.ru/20240112/evro-1920866381.html,Курс евро на Московской бирже опустился ниже 9...,Курс евро на Московской бирже опустился ниже 9...,"МОСКВА, 12 янв — РИА Новости. Российская валют...",10:05 12.01.2024
97,1920862656,https://ria.ru/20240112/zerno-1920862656.html,В России не планируют отменять экспортные пошл...,Абрамченко: власти не планируют отменять экспо...,"МОСКВА, 12 янв - РИА Новости. Власти РФ не пла...",09:35 12.01.2024


Добавьте в таблицу `df` столбец `topic`, содержащий список `topics_order_fixed`

In [14]:
df["topic"] = topics_order_fixed
df

,id,url,title,subtitle,content,datetime,topic
0,1921570252,https://ria.ru/20240116/pivo-1921570252.html,В ГД оценили идею запрета ввоза импортного пива,В ГД назвали идею запрета ввоза импортного пив...,"МОСКВА, 16 янв – РИА Новости. Депутат Госдумы ...",11:17 16.01.2024,economy
1,1921562909,https://ria.ru/20240116/pivo-1921562909.html,В Госдуме предложили запретить ввоз пива из не...,"""Известия"": в РФ хотят запретить поставки пива...","МОСКВА, 16 янв – РИА Новости. Глава комитета Г...",10:40 16.01.2024,economy
2,1921490766,https://ria.ru/20240116/neft-1921490766.html,Голод не тетка: CША накупили российской нефти ...,США возобновили импорт нефти из России после п...,"МОСКВА, 16 янв — РИА Новости, Наталья Дембинск...",08:00 16.01.2024,economy
3,1921539497,https://ria.ru/20240116/bortprovodniki-1921539...,"""Аэрофлот"" и ""Победа"" готовы принять на работу...","""Аэрофлот"" и ""Победа"" готовы принять на работу...","МОСКВА, 16 янв - РИА Новости. Крупнейшая росси...",07:24 16.01.2024,economy
4,1921529559,https://ria.ru/20240116/platina-1921529559.html,Германия сократила закупки платины у России до...,ФРГ в ноябре сократила закупки платины у Росси...,"МОСКВА, 16 янв - РИА Новости. Германия к концу...",03:05 16.01.2024,economy
...,...,...,...,...,...,...,...
94,1920886637,https://ria.ru/20240112/minpromtorg-1920886637...,Минпромторг разработал проект эксперимента по ...,Минпромторг: в России предложили маркировать р...,"МОСКВА, 12 янв - РИА Новости. Минпромторг Росс...",11:23 12.01.2024,economy
95,1920878136,https://ria.ru/20240112/moldaviya-1920878136.html,"Додон заявил, что власти Молдавии все равно по...",Додон: власти Молдавии покупают тот же российс...,"КИШИНЕВ, 12 янв – РИА Новости. Экс-президент М...",10:56 12.01.2024,economy
96,1920866381,https://ria.ru/20240112/evro-1920866381.html,Курс евро на Московской бирже опустился ниже 9...,Курс евро на Московской бирже опустился ниже 9...,"МОСКВА, 12 янв — РИА Новости. Российская валют...",10:05 12.01.2024,economy
97,1920862656,https://ria.ru/20240112/zerno-1920862656.html,В России не планируют отменять экспортные пошл...,Абрамченко: власти не планируют отменять экспо...,"МОСКВА, 12 янв - РИА Новости. Власти РФ не пла...",09:35 12.01.2024,economy


## Вопрос 3

Какой `topic` имеет новость из первой строки таблицы?

In [15]:
df.iloc[0]

id                                                 1921570252
url              https://ria.ru/20240116/pivo-1921570252.html
title         В ГД оценили идею запрета ввоза импортного пива
subtitle    В ГД назвали идею запрета ввоза импортного пив...
content     МОСКВА, 16 янв – РИА Новости. Депутат Госдумы ...
datetime                                     11:17 16.01.2024
topic                                                 economy
Name: 0, dtype: object

Сохраним скачанные данные в файл.

In [16]:
df.to_pickle("df_ria.p", compression="gzip")

## Вопрос 4

Есть ли пропуски в каких-либо столбцах в таблице `df`?

In [17]:
df.isnull().sum()

id          0
url         0
title       0
subtitle    0
content     0
datetime    0
topic       0
dtype: int64